In [401]:
#imports
import boto3
from botocore.exceptions import ClientError #for debugging
from dotenv import load_dotenv #for env variables
import os
import pandas as pd
from sqlalchemy import create_engine
import psycopg2


In [402]:
#load the variables/password/IDs from the .env file
load_dotenv()
#get the access and secret key 
access_key_id = os.getenv('ACCESS_KEY_ID')
secret_key = os.getenv('SECRET_ACCESS_KEY')

#declare bucket name and data folder name where data will be downloaded to
bucket_name = "mindex-data-analytics-code-challenge"
data_folder = "/Users/mady/mindex_data_analytics_challenge/data/"


In [403]:
#create s3 client
s3 = boto3.client('s3', aws_access_key_id=access_key_id, aws_secret_access_key=secret_key, region_name='us-east-1')

#file names to download
files = ['bengals.csv', 'boyd_receiving.csv', 'chase_receiving.csv', 'higgins_receiving.csv']

#download each file
for file in files:
    try:
        s3.download_file(bucket_name, file, data_folder + file)
        print(file + " downloaded successfully")
    except Exception as e:
        print(file + "error: ", e)

bengals.csv downloaded successfully
boyd_receiving.csv downloaded successfully
chase_receiving.csv downloaded successfully
higgins_receiving.csv downloaded successfully


In [404]:
'''
Use the pandas library to load each CSV into its own dataframe.
'''
dataframes = {} #dictionary to hold the dataframes for each player
player_files = ['/Users/mady/mindex_data_analytics_challenge/data/boyd_receiving.csv', '/Users/mady/mindex_data_analytics_challenge/data/chase_receiving.csv', '/Users/mady/mindex_data_analytics_challenge/data/higgins_receiving.csv']

#put each file into a pandas dataframe
for file in player_files:
    csv = file.split('/data/')[1]
    player_name = csv.split('_')[0].capitalize()
    print(player_name + " dataframe created")
    dataframes[player_name] = pd.read_csv(file)
    #add player name as a column
    dataframes[player_name]['Player'] = player_name

#print out dataframe heads just created
print('****** Boyd df ****** \n ', dataframes['Boyd'].head())
print('****** Higgins df ****** \n ',dataframes['Higgins'].head())
print('****** Chase df ****** \n ',dataframes['Chase'].head())

#load bengals data into a panda frame
dataframes['Bangals'] = pd.read_csv('/Users/mady/mindex_data_analytics_challenge/data/bengals.csv')

Boyd dataframe created
Chase dataframe created
Higgins dataframe created
****** Boyd df ****** 
     Week  Yards  TD Player
0  REG1     32   0   Boyd
1  REG2     73   0   Boyd
2  REG3     36   1   Boyd
3  REG4    118   0   Boyd
4  REG5     24   0   Boyd
****** Higgins df ****** 
     Week  Yards  TD   Player
0  REG1     58   1  Higgins
1  REG2     60   1  Higgins
2  REG5     32   0  Higgins
3  REG6     44   0  Higgins
4  REG7     62   0  Higgins
****** Chase df ****** 
     Week  Yards  TD Player
0  REG1    101   1  Chase
1  REG2     54   1  Chase
2  REG3     65   2  Chase
3  REG4     77   0  Chase
4  REG5    159   1  Chase


In [405]:
'''
Join/Merge all of the dataframes together to display one global table that shows
the three different receiver’s yards and touchdown (TD) data as well as every
game result. Be sure to include Opponent, Location, and Result fields from the
bengals.csv file.

Replace the ‘1.0’ or ‘0.0’ values in the Result field to display ‘Win’ or ‘Loss’,
respectively.
'''

#concat the player data frames
df = pd.concat([dataframes['Boyd'], dataframes['Higgins'], dataframes['Chase']])

#join to the bengals data frame
df = df.merge(dataframes['Bangals'], on='Week', how= 'right')
df[['Yards', 'TD']] = df[['Yards', 'TD']].fillna(0)

#replace the 1.0 and 0.0 values in the Result field
df = df.replace({'Result': {1: 'Win', 0: 'Loss'}})
df.index = range(1, len(df) + 1)

#add indexs column so it's in CSV
df['Index'] = range(1, len(df) + 1)

#replace empty yard values with 0 (int)
df['Yards'] = df['Yards'].replace('', 0).fillna(0).astype(int)
df['TD'] = df['Yards'].replace('', 0).fillna(0).astype(int)

#save the global dataframe to a csv file 
df.to_csv('/Users/mady/mindex_data_analytics_challenge/data/global.csv', index=False)

In [406]:
#get variaables from the .env file
db_host = os.getenv('DB_HOST')
db_username = os.getenv('USERNAME')
db_password = os.getenv('PASSWORD')
db_name = os.getenv('DB_NAME')
db_table_name = os.getenv('TABLE_NAME')
db_address = os.getenv('ADDRESS')

#db connection details 
#note address is the IP address from terminal running nslookup and the host given
#because the host was not working to let me connect
db = {
    'dbname': db_name,
    'user': db_username,
    'password': db_password,
    'host': db_address,
    'port': '5432'
}
connection_string = 'postgresql+psycopg2://' + db_username + ':' + db_password + '@' + db_address + '/' + db_name
#create SQLalchemy engine
engine = create_engine(connection_string)

#for debugging, prints my table
def print_my_table():
    try:
        with engine.connect() as connection:
            query = f"SELECT * FROM {db_table_name};"
            mb = pd.read_sql(query, con=connection.connection)
            print("connection to DB successful, table to print below")
            print(mb)

    except Exception as e:
        print("error connecting to PostgreSQL DB: ", e)
        

In [407]:
#helper method to execute a query 
def execute_query_helper(query):
    try:
        conn = psycopg2.connect(**db)
        cursor = conn.cursor()
        alter_query = query
        cursor.execute(alter_query)
        conn.commit()

    except Exception as e:
        print(f"Error: {e}")
    finally:
        #close connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#delete contents from table 
execute_query_helper("""DROP VIEW if exists results;""")
execute_query_helper(""" DELETE FROM madison_banaszak; """)

#remove columns
execute_query_helper(""" ALTER TABLE madison_banaszak
                     DROP COLUMN Week,
        DROP COLUMN Yards,
        DROP COLUMN TD,
        DROP COLUMN Player,
        DROP COLUMN Opponent,
        DROP COLUMN Location,
        DROP COLUMN Result,
        DROP COLUMN Index;
                     """)
#add columms back in
execute_query_helper("""  
        ALTER TABLE madison_banaszak
        ADD COLUMN Week TEXT,
        ADD COLUMN Yards INT,
        ADD COLUMN TD INT,
        ADD COLUMN Player TEXT,
        ADD COLUMN Opponent TEXT,
        ADD COLUMN Location TEXT,
        ADD COLUMN Result TEXT,
        ADD COLUMN Index INT;
                     """)
print_my_table()


connection to DB successful, table to print below
Empty DataFrame
Columns: [week, yards, td, player, opponent, location, result, index]
Index: []


/var/folders/m_/b_wfgwbs4bz_t1zz98v8qgfr0000gn/T/ipykernel_19023/2712470670.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mb = pd.read_sql(query, con=connection.connection)


In [ ]:
#get connection to db

#delete contents from db before insert
execute_query_helper(""" DELETE FROM madison_banaszak; """)
conn = psycopg2.connect(**db)
#create cursor object
curr = conn.cursor()

#open the csv file
with open('/Users/mady/mindex_data_analytics_challenge/data/global.csv', 'r') as f:
    #skip the header row
    next(f)
    #copy the csv file into the table
    curr.copy_from(f, 'madison_banaszak', sep=',')
    conn.commit()
    curr.close()
    conn.close()
    print("CSV file copied into table successfully :)")

print_my_table()


about to clean out table first before insert
connection to DB successful, table to print below
Empty DataFrame
Columns: [week, yards, td, player, opponent, location, result, index]
Index: []
CSV file copied into table successfully :)


/var/folders/m_/b_wfgwbs4bz_t1zz98v8qgfr0000gn/T/ipykernel_19023/2712470670.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mb = pd.read_sql(query, con=connection.connection)
/var/folders/m_/b_wfgwbs4bz_t1zz98v8qgfr0000gn/T/ipykernel_19023/2712470670.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mb = pd.read_sql(query, con=connection.connection)


connection to DB successful, table to print below
     week  yards   td   player opponent location result  index
0    PRE1      0    0                TB     Away    Win      1
1    PRE2      0    0               WSH     Away   Loss      2
2    PRE3      0    0               MIA     Home   Loss      3
3    REG1     32   32     Boyd      MIN     Home    Win      4
4    REG1     58   58  Higgins      MIN     Home    Win      5
..    ...    ...  ...      ...      ...      ...    ...    ...
58  POST3    103  103  Higgins       KC     Away    Win     59
59  POST3     54   54    Chase       KC     Away    Win     60
60  POST4     48   48     Boyd      LAR  Neutral   Loss     61
61  POST4    100  100  Higgins      LAR  Neutral   Loss     62
62  POST4     89   89    Chase      LAR  Neutral   Loss     63

[63 rows x 8 columns]


In [409]:
'''
Set up a connection to view the database using DBeaver and ensure all relevant
data is present. Write a SQL Query to show the total yards each receiver had
throughout the season as well as the team's record displayed as: “# of Wins - #
of Losses”. Your query should generate the following view:
'''
def print_select_helper(query):
    try:
        conn = psycopg2.connect(**db)
        cursor = conn.cursor()
        alter_query = query
        cursor.execute(alter_query)
        s = pd.read_sql(alter_query, conn)
        print(s)
        return s
                 
    except Exception as e:
        print(f"Error: {e}")
    finally:
        #close connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()


execute_query_helper("""DROP VIEW if exists results;""")
execute_query_helper(""" CREATE OR REPLACE VIEW results as
WITH yards AS (
    SELECT
        SUM(CASE WHEN Player = 'Boyd' THEN Yards ELSE 0 END) AS "Boyd Yards",
        SUM(CASE WHEN Player = 'Higgins' THEN Yards ELSE 0 END) AS "Higgins Yards",
        SUM(CASE WHEN Player = 'Chase' THEN Yards ELSE 0 END) AS "Chase Yards"
    FROM madison_banaszak
),
wins_losses AS (
  SELECT
 CONCAT(
        SUM(CASE WHEN Result = 'Win' THEN 1 ELSE 0 END), 
        ' - ', 
        SUM(CASE WHEN Result = 'Loss' THEN 1 ELSE 0 END)
    ) AS "Wins/Losses"
FROM
    (SELECT DISTINCT Week, Result FROM madison_banaszak) AS distinct_results
    
)
SELECT * 
FROM  yards cross join wins_losses;

                     """)

print_select_helper(""" SELECT * FROM results; """) 

   Boyd Yards  Higgins Yards  Chase Yards Wins/Losses
0         938           1400         1823     14 - 10


/var/folders/m_/b_wfgwbs4bz_t1zz98v8qgfr0000gn/T/ipykernel_19023/3777064514.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(alter_query, conn)


,Boyd Yards,Higgins Yards,Chase Yards,Wins/Losses
0,938,1400,1823,14 - 10
